In [2]:
#need to read in a layer
#test a probe and write the results 

#multiple layers? --> last three of hte xl embeddings 

In [7]:
import pandas as pd
df = pd.read_csv('./data/animals_true_false.csv')
source_labels = df['label']

In [3]:
import os
import numpy as np
import re

# Set the folder path
folder_path = "data/embeddings/llama3/basic"
llama_embeddings = []

# Regular expression pattern to match negative numbers
pattern = r'-(\d+)'

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    # Check if the file is a numpy file
    if filename.endswith(".npy"):
        # Load the embedding data from the file
        embedding_data = np.load(file_path)

        # Search for the negative number in the filename
        match = re.search(pattern, filename)
        if match:
            layer = int(match.group(1))

            # Initialize the list for the current layer if necessary
            if len(llama_embeddings) <= layer:
                llama_embeddings.extend([[] for _ in range(layer - len(llama_embeddings) + 1)])

            # Append the embedding data to the corresponding layer
            llama_embeddings[layer].append(embedding_data)

In [3]:
from itertools import combinations

def generate_combinations(x, y):
    """
    Generates all combinations of numbers from x to y.
    
    Args:
        x (int): The lower bound of the range (inclusive).
        y (int): The upper bound of the range (inclusive).
        
    Returns:
        list: A list of tuples, where each tuple represents a combination
        of numbers from x to y.
    """
    nums = list(range(x, y+1))  # Generate a list of numbers from x to y
    combinations_list = []
    
    # Generate combinations of different lengths
    for length in range(1, len(nums)+1):
        combs = combinations(nums, length)
        combinations_list.extend(combs)
    
    return combinations_list

In [19]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def probe(embeddings, labels, layer_list, input_dim=4096):
  X = np.array(embeddings)  
  y = np.array(labels)

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Define and compile model
  model = Sequential()
  model.add(Dense(256, activation='relu', input_dim=input_dim))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  # Train the model
  model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

  # Evaluate the model
  test_loss, test_acc = model.evaluate(X_test, y_test)
  predictions = model.predict(X_test) #predictions
  print(predictions)
  

  layer_str = '-'.join(map(str, layer_list))
  save_data(test_loss, test_acc, layer_str)

def save_data(test_loss, test_acc, layer):
  write_data = f'{layer}: {test_loss} {test_acc}'
  with open('./statistics/Layer_Data/replicate.txt', 'a') as file:
    file.write(f'{write_data}\n')

In [19]:
indices = generate_combinations(0,48)

In [16]:
indices

[(47,), (48,), (47, 48)]

In [7]:
def parse_embeddings(embeddings_list, index_list):
    flat_list = []
    for index in index_list:
        flat_list.extend(embeddings_list[index])
    return flat_list  

In [71]:
indices = [(1, 2)]

In [72]:
embedding_data = llama_embeddings[-1] + llama_embeddings[-2][:1008]

for index_list in indices:
  #construct embeddings for this list 
  #create the label amount by multiplying 
  embeddings = parse_embeddings(basic_embeddings, index_list)
  labels = df['label'] * len(index_list)
  probe(embeddings=embedding_data, labels=labels, layer_list=index_list)

ValueError: Found input variables with inconsistent numbers of samples: [2016, 1008]

In [79]:
list = df['label'].to_list()
list.extend(df['label'].to_list())
list.extend(df['label'].to_list())


probe(embeddings=llama_embeddings[-1] + llama_embeddings[-2][:1008] + llama_embeddings[-3][:1008], labels=list, layer_list=[123])

Epoch 1/10


/Users/ayushjain/Development/Personal-Projects/Probing/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5211 - loss: 0.9647 - val_accuracy: 0.5207 - val_loss: 0.7873
Epoch 2/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5118 - loss: 0.7220 - val_accuracy: 0.5041 - val_loss: 0.7325
Epoch 3/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5395 - loss: 0.6996 - val_accuracy: 0.4587 - val_loss: 0.7522
Epoch 4/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5763 - loss: 0.6856 - val_accuracy: 0.4525 - val_loss: 0.7305
Epoch 5/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5737 - loss: 0.6733 - val_accuracy: 0.4711 - val_loss: 0.7444
Epoch 6/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5814 - loss: 0.6768 - val_accuracy: 0.5269 - val_loss: 0.8133
Epoch 7/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5525 - loss: 0.7086 - val_accuracy: 0.5248 - val_loss: 0.7294
Epoch 8/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6037 - loss: 0.6533 - val_accuracy: 0.4669 - val_loss: 0.7500
Epo

In [36]:
llama_embeddings[-1]

[array([ 0.01747116, -0.42323282,  0.30830935, ...,  0.46633828,
        -0.12056616,  0.09740624], dtype=float32),
 array([ 0.12239196, -0.63870466,  0.02683339, ..., -0.32420284,
        -0.8566279 ,  0.20964763], dtype=float32),
 array([ 0.05546749, -0.48597947, -0.08816382, ...,  0.48524636,
        -0.18870753,  0.01829191], dtype=float32),
 array([-0.12096751, -0.5143766 ,  0.4060651 , ..., -0.39222383,
        -0.5861286 ,  0.05825946], dtype=float32),
 array([-0.67414135, -1.0228579 , -0.37568468, ..., -0.10261825,
         0.12152176,  0.31148466], dtype=float32),
 array([-0.00646889, -0.29398885,  0.1867952 , ..., -0.04239854,
        -0.7892389 , -0.19945805], dtype=float32),
 array([-0.11910202, -0.28407276, -0.30232972, ...,  0.42460865,
        -0.715851  ,  0.12741405], dtype=float32),
 array([ 0.51039314, -0.5164501 ,  0.6577026 , ..., -0.632338  ,
        -0.9324554 ,  0.17022057], dtype=float32),
 array([-0.17046036, -0.65159345, -0.2388604 , ...,  0.09300896,
       

# Subspace Theory

In [80]:
first_layer = llama_embeddings[-1]
second_layer = llama_embeddings[-2]
third_layer = llama_embeddings[-3]

In [82]:
len(first_layer[0]) #4096 dimension embedding

4096

### PCA